# 验证和授权概述
- django有一个内置的授权系统，它是来处理用户、分组、权限以及基于cookie的会话系统。django的授权系统包括验证和授权两个部分，验证时验证这个用户是否是它声称的人（比如用户名和密码），授权是给他相应的权限。django内置的权限系统包括以下方面：
    - 1.用户
    - 2.权限
    - 3.分组
    - 4.一个可以配置密码的哈希系统
    - 5.一个可插拔的后台管理系统

# 使用授权系统
- 默认创建完一个django项目后，就集成了授权系统。
- 与授权系统相关的INSTALLED_APPS
    - 1.django.contrib.auth:包含了一个核心授权框架，以及大部分的模型定义
    - 2.django.contrib.contenttypes:Content Type系统，可以用来关联模型和权限

# 中间件
- 1.SessionMiddleware：用来管理session
- 2.AuthenticationMiddleware：用来处理和当前session相关联的用户

# User模型
- User模型是这个框架的核心部分，它完整的路径是在django.contrib.models.User

## 字段
- 内置的User模型有以下字段
    - 1.username：用户名：150个字以内，可以包含数字和英文字符，以及_、@、+、-、.字符，不能为空，且必须唯一
    - 2.first_name：名
    - 3.last_name：姓氏
    - 4.email：邮箱，可以为空
    - 5.password：密码，经过哈希后的密码
    - 6.groups：分组，一个用户可以属于多个分组，一个分组可以拥有多个用户，groups这个字段是跟Group的一个多对多的关系
    - 7.user_permissions：权限，一个用户可以拥有多个权限，一个权限可以被多个用户所拥有，和Permission属于一种多对多的关系
    - 8.is_staff：是否可以进入到admin的站点（表示是否是员工）
    - 9.is_acitve：是否是可用的，对于一些想要删除账号的数据，我们设置这个值为False就可以了，而不是真正的从数据库中删除
    - 10.is_superuser：是否是超级管理员，如果是，那么拥有整个网站的所有权限
    - 11.last_login：上次登录的时间
    - 12.date_joined：账号创建的时间
    
    
## User模型的基本用法
### 创建用户：
- 导入：from django.contrib.auth.models import User,使用User.objects.create()
- 通过create_user方法可以快速的创建用户，这个方法必须要传递username，email，password参数

### 创建超级用户
- 使用：User.objects.create_superuser()，个方法必须要传递username，email，password参数
- 也可以在终端创建:python manage.py createsuperuser

### 修改密码
- 密码不能通过直接赋值的方式进行修改，因为它存储在数据库中是经过了哈希的
- 如下：
        user = User.objects.get(pk=1)
        ser.set_password("new_password")
        user.save()
        
### 登录验证
- django的验证系统已经实现了登录验证的功能，- django内置有一个函数来验证用户的登录：内置的这个方法只能来验证用户名和密码
        from django.contrib.auth import authenticate
        user = authenticate(request, username=, password=)
- 如果想要自己定义验证哪写字段，可以自己定义一个函数。还有一种方法是继承AbstractUser时定义属性：USERNAME_FIELD = "指定的字段名"，在使用authenticate的时候这样用：user = authenticate(request, username="指定的字段名的值", password=password")。注意username属性我们并没有改变它，所以只能使用它
        def myauthentic(telephone, password):
            # 通过子表中的telephone字段来进行筛选
            user = User.objects.filter(extension__telephone=telephone).first()
            # 先判断数据库中是否有这个电话号码
            if user:
                # 如果有号码，就检查密码是否正确，若正确就返回user对象
                is_correct = user.check_password(password)
                if is_correct:
                    return user
                else:
                    return None
            else:
                return None
        
            # 此时是修改USERNAME_FIELD之后使用authentic
            telephone = request.GET.get("telephone")
            password = request.GET.get("password")
            # 注意，并没有修改authenticate的username这个参数的名称
            user = authenticate(request, username=telephone, password=password)
            if user:
                return HttpResponse("恭喜你登录成功")
            else:
                return HttpResponse("号码或密码有误")

# 扩展用户模型
- django内置的User模型虽然已经很强大了，但是有时候还是不能满足我们的需求，比如验证用户登录的时候，它使用的是用户名作为验证，而我们有时候需要使用手机号码或者邮箱来验证，再比如我们想增加一些新的字段，那么就需要模型扩展。扩展用户模型有多种方式。


## 1.设置Proxy模型
- 如果你对django提供的字段，以及验证的方法都比较满意，没什么需要修改，但是只是需要在它原有的基础之上增加一些操作的方法，那么可以使用这个方式
- 具体代码见django_authentic
- 示例：
        # 定义一个代理模型,代理模型可以扩展操作，但是不能新增字段，会报错
        # 主要是在Meta中进行扩展操作
        class Person(User):
            class Meta:
                proxy = True

            @classmethod
            # 比如想要获取is_active为0的用户
            # 为啥不直接通过User对象直接进行过滤操作呢，可以是可以，复用比较麻烦
            def get_blacklist(cls):
                return cls.objects.filter(is_active=False)
        
        # views.py中的代码
        def get_blacklist(request):
            people = Person.get_blacklist()
            for person in people:
                print(person.username)
            return HttpResponse("获取成功")
- 以上代码，我们定义了一个Person类，继承自User，并且在Meta中设置proxy=True，说明这个只是user的一个代理模型。它并不会影响原来的User模型在数据库中的结构，以后如果你想方便的获取所有黑名单的人，就可以通过Person.get_balcklist()就可以获取到。并且User.objects.all()和Person.objects.all()其实是等价的。
        
## 2.一对一外键：
- 如果你对用户验证方法authentic没有其他要求，就是使用username和password来完成，但是想要在原来模型的基础之上增加新的字段，那么可以使用一对一外键的方式。定义一对一外键除了要建立一对一直接的外键关系外，还要使用reveiver来实现两表之间的联动，详情见下面
- 具体代码见django_authentic
- 代码示例：
        # 使用一对一外键关系来扩充User对象的字段
        class UserExtension(models.Model):
            user = models.OneToOneField(User, on_delete=models.CASCADE, related_name="extension")
            telephone = models.CharField(max_length=11)
            school = models.CharField(max_length=100)


        # 由于他们是一对一关系，所以单纯的靠这个关系无法实现联动，比如User对象修改某个属性什么的
        # 所以必须要添加一个信号来实现User和UserExtension的联动.以下是固定写法
        from django.dispatch import receiver
        from django.db.models.signals import post_save
        @receiver(post_save, sender=User)
        def handler_user_extension(sender, instance, created, **kwargs):
            if created:
                UserExtension.objects.create(user=instance)
            else:
                instance.extension.save()
        # ----------------------------------------------------------------------------------      
        # views.py文件中的代码
        # 新增字段以后，就自己定义验证函数来验证自己新增的字段
        def myauthentic(telephone, password):
            # 通过子表中的telephone字段来进行筛选
            user = User.objects.filter(extension__telephone=telephone).first()
            # 先判断数据库中是否有这个电话号码
            if user:
                # 如果有号码，就检查密码是否正确，若正确就返回user对象
                is_correct = user.check_password(password)
                if is_correct:
                    return user
                else:
                    return None
            else:
                return None
                
        # 使用一对一外键关系构建的,通过  user对象.子表小写名.新增字段  的方式来访问User子表的字段，相当于给User扩充了字段
        def one_view(request):
            # user_songsong = User.objects.create_user(username="songsong", email="123@qq.com", password="123456")
            user = User.objects.get(username="songsong")
            userextension = user.extension
            userextension.telephone = "18276457813"
            userextension.school = "武汉科技大学"
            user.save()
            return HttpResponse("创建成功")

- 注：以上两个详情代码见django_authentic
- 由于继承AbstractUser需要在最开始对表迁移的时候就进行设置，所以新建一个数据库，代码见django_abstractuser
## 3.继承自AbstractUser
- 对于authentic不满意，并且不想要修改User对象上的一些字段，但是想要增加一些字段，此时就可以直接继承自django.contrib.auth.AbstractUser,其实这个类也是django.contrib.auth.models.User的父类
- 这种方式因为破坏了原来User模型的表结构，所以必须要在第一次迁移之前就定义好。该方法只是对User部分重构
- 具体代码见django.abstractuser
- 操作步骤：
    - 1.要继承AbstractUser就必须要重写User模型来覆盖内置的User，同时在settings.py中设置AUTH_USER_MODEL 
            # 这是告知系统User在哪个APP中，直接 APP名字.User 即可，不用APP名字.models.User
            AUTH_USER_MODEL = "APP名字.User"

    - 2.在models中书写代码如下：
            from django.db import models
            from django.contrib.auth.models import AbstractUser, BaseUserManager

            # 定义自己的manger，它的作用是让在创建用户时，规定哪些参数必须要传入，其他的随便你传不传
            # 在这里面可以重写创建用户的create_user和create_superuser方法
            class UserManager(BaseUserManager):
                def _create_user(self, telephone, username, password, **kwargs):
                    if not telephone:
                        # 如果创建用户的时候没有传入号码就给他报错
                        raise ValueError("号码不能为空")
                    if not password:
                        raise ValueError("密码不能为空")
                    if not username:
                        raise ValueError("用户名不能为空")
                    # self.model指向的是自己定义的User模型
                    user = self.model(telephone=telephone, username=username, **kwargs)
                    user.set_password(password)
                    user.save()
                    return user

                # 重写创建普通用户的函数
                def create_user(self, telephone, username, password, **kwargs):
                    # 普通用户的is_superuser的属性为False
                    kwargs['is_superuser'] = False
                    return self._create_user(telephone=telephone, username=username, password=password, **kwargs)

                # 重写创建超级用户的方法
                def create_superuser(self, telephone, username, password, **kwargs):
                    kwargs['is_superuser'] = True
                    return self._create_user(telephone=telephone, username=username, password=password, **kwargs)



            # 要继承AbstractUser的话就必须要把User覆盖掉
            # 同时在settings设置AUTH_USER_MODEL = "front.User"
            class User(AbstractUser):
                telephone = models.CharField(max_length=11, unique=True)
                school = models.CharField(max_length=100)

                USERNAME_FIELD = "telephone"
                # 这句话的意思是在创建对象的时候telephone就成为authentic函数中的那个username参数
                # 即authentic进行判断的时候传入的不再是用户名，而是telephone进行验证判断（这个值必须要是唯一的）
                objects = UserManager()
                # 将objects指向自己设定的UserManager，这样才能才能生效
                # objects就是manager类型

## 4.继承自AbstractBaseUser模型
- 如果想要修改默认的验证方式，并且对于原来User模型上的一些自带的字段不需要（比如first_naem ，last_name），那么就可以定义一个模型，然后继承自AbstractBaseUser，再添加你想要的字段，这种方式会很麻烦，最好是确定自己对django比较了解才推荐使用。步骤如下：
- 具体代码见django_abstractbaseuser
    - 在model.py中创建模型，然后在settings中设置AUTH_USER_MODEL
            from django.db import models
            from django.contrib.auth.models import BaseUserManager, AbstractBaseUser, PermissionsMixin
            from django.contrib.auth import get_user_model

            # Create your models here.
            # 定义自己的manger，它的作用是让在创建用户时，规定哪些参数必须要传入，其他的随便你传不传
            # 在这里面可以重写创建用户的create_user和create_superuser方法
            class UserManager(BaseUserManager):
                def _create_user(self, telephone, username, password, **kwargs):
                    if not telephone:
                        # 如果创建用户的时候没有传入号码就给他报错
                        raise ValueError("号码不能为空")
                    if not password:
                        raise ValueError("密码不能为空")
                    if not username:
                        raise ValueError("用户名不能为空")
                    # self.model指向的是自己定义的User模型
                    user = self.model(telephone=telephone, username=username, **kwargs)
                    user.set_password(password)  #密码要单独进行设置，否则存储的密码就是明文
                    user.save()
                    return user

                # 重写创建普通用户的函数
                def create_user(self, telephone, username, password, **kwargs):
                    # 普通用户的is_superuser的属性为False
                    kwargs['is_superuser'] = False
                    return self._create_user(telephone=telephone, username=username, password=password, **kwargs)

                # 重写创建超级用户的方法
                def create_superuser(self, telephone, username, password, **kwargs):
                    kwargs['is_superuser'] = True
                    return self._create_user(telephone=telephone, username=username, password=password, **kwargs)


            # 算是彻底重写User,他要继承两个
            class User(AbstractBaseUser, PermissionsMixin):
                # 定义自己想要的字段，但是不要定义password字段
                # password不要重写了，因为它本身涉及到哈希算法的加密，你可能写不出来
                telephone = models.CharField(max_length=11, unique=True)  #用来验证的字段必须得是唯一的
                email = models.CharField(max_length=100, unique=True)
                username = models.CharField(max_length=100)
                is_active = models.BooleanField(default=True)  # 该属性必须要加上，否则后面用不了

                # 定义用来验证的字段，和abstract案例一样，见它的代码
                USERNAME_FIELD = 'telephone'
                # 定义在创建超级用户时应该要传递哪些参数，默认是传递两个，即你自己定义的字段和密码，
                # 若想让在创建时传递更多的参数，把它的字段名放在括号中
                REQUIRED_FIELDS = []

                def get_full_name(self):
                    return self.username

                def get_short_name(self):
                    return self.username
                # 重写方法时尽量按照它原来代码中的写法来写，避免出错

                objects = UserManager()

            class Article(models.Model):
                title = models.CharField(max_length=100)
                # author = models.ForeignKey(User, on_delete=models.CASCADE)
                # 当要创建外键关联关系时，如果在同一个模型中确实可以直接调用，但是如果在另外一个模型中想要使用User这种方法就不行了
                # 应该使用如下的方法:导入：from django.contrib.auth import get_user_model
                # 直接获取你在settings.py中配置的AUTH_USER_MODEL = 'front.User'对象
                author = models.ForeignKey(get_user_model(), on_delete=models.CASCADE)

### 注意：无论是AbstractUser还是AbstractBaseUser，在第一次迁移之前，一定要先在models.py文件中定义好User模型